以下の論文をD-Waveを利用して解いていく

https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full

日本語の詳しいサイトです

https://quantum.fixstars.com/introduction_to_quantum_computer/quantum_computer_research/np_problem_ising/

In [ ]:
%matplotlib inline
from sympy import *
init_printing()
from sympy.printing.mathml import print_mathml

# Partitioning Problems(分割問題)

## 整数分割問題

### 整数分割問題とは
自然数分割問題とは2つのグループの和が等しくなるように自然数を分けるものである。
自然数のグループを$q_i$で表してみる。1つめのグループを$q_1$、2つめのグループを$q_2$とすると$q_1$と$q_2$のグループの
自然数の和が等しいときに最小になるようなコスト関数Eを考える。

$E =  f(x) = \{ \sum_{i=1}^N n_i \times (2q_i - 1)  \}^2$

上記のようにすると、
- 自然数$n_i$がグループAに属するときは$2q_i - 1 = 1$
- 自然数$n_i$がグループBに属するときは$2q_i - 1 = -1$

従って、双方のグループの和が等しい場合に$E = 1$となる。

In [ ]:
E, i, N, q_i,j = symbols('E i N q_i j')
n_i = Symbol('n_i')
#n_i = numbered_symbols('n') # うまく行かない
E = Sum(n_i*(2*q_i -1),(i,1,N))**2
ex = E.expand(basic=True)
ex

$(\sum_{i=1}^N - n_i)^2$の項は最小関数を求めるので不要である。また、全体を4で割る。

In [ ]:
q_j,n_j = symbols('q_j n_j')
Sum(n_i*q_i, (i, 1, N))**2 - Sum(n_i, (i, 1, N))*Sum(n_j*q_j, (j, 1, N))

$q_i = 0$または$q_i = 1$のときには$q_i^2=q_i$である。また、$\sum_{i=1}^N n_i $はnの総和であり、$n_s$ とする。

In [ ]:
n_sum = Symbol('n_sum')
Sum(n_i*q_i, (i, 1, N))**2 - n_sum*Sum(n_j*q_j, (j, 1, N))



　$E=\sum_{i=1}^{N}(n_i^2 - n_{sum}n_i)q_i +2 \sum_{i < j}n_in_jq_iq_j$ 


　$qubo = \left[\begin{array}{rrrrr}n_1^2 - n_{sum}n_1 & 2n_1n_2 & 2n_1n_3 & 2n_1n_4 & ...\\ 0 & n_2^2 - n_{sum}n_2 & 2n_2n_3& 2n_2n_4 &...\\ 0 & 0 & n_3^2 - n_{sum}n_3 & 2n_3n_4 & ...\\ 0 & 0 & 0 & n_4^2 - n_{sum}n_4 & ...\\ ... & ... & ... & ... &... \end{array} \right]$ 

In [ ]:
def get_results(result, max):
    r = []
    R = iter(result)
    E = iter(result.data())
    print(len(result))
    i = 0
    for line in result:
        sample = next(R)
        data = next(E)
        energy = data.energy
        occurrences = data.num_occurrences
        r.append([energy, sample,occurrences])

        i = i + 1
        if i >= max:
            break
        #print(f'{energy},{sample}, {occurrences}')
    return r

def divide_groups(sample):
    keys = sample.keys()
    arr = [0] * len(keys)
    for key in keys:
        arr[key] = sample[key]
    return arr

In [ ]:
import dwave.inspector
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import numpy as np

N = 5
np.random.seed(8123172)
arr = np.random.randint(1, N, N)

Q = {}
# numbers = [3,2,6,9,2,5,7,3,3,6,7,3,5,3,2,2,2,6,8,4,6,3,3,6,4,3,3,2,2,5,8,9]
numbers = arr.tolist()
sum_n = sum(numbers)
for i in range(len(numbers)):
  for j in range(len(numbers)):
    if i == j:
      Q[i,j]=numbers[i]**2-sum_n*numbers[i]
    elif i<j:
      Q[i,j]=2*numbers[i] * numbers[j]
    
print('setup qubo finished.....')
sampler = EmbeddingComposite(DWaveSampler(qpu='Advantage_system1.1')) 
print('enbedding finished.....')
result = sampler.sample_qubo(Q, chain_strength=3, num_reads=1000)
print('get result from d-wave.....')

In [ ]:
print(arr)
ret = get_results(result,5)
for item in ret:
    ans = divide_groups(item[1])
    print(ans)
    r = range(len(ans))
    group_1 = []
    group_2 = []
    for i in r:
        if ans[i]==0:
            group_1.append(numbers[i])
        else:
            group_2.append(numbers[i])
    # print(item[0],len(group_1),sum(group_1),len(group_2),sum(group_2))
    print('no of elements in group_1=',len(group_1),'sum in group_1=',sum(group_1))
    print('no of elements in group_2=',len(group_2),'sum in group_2=',sum(group_2))

In [ ]:
#dwave.inspector.show(result) 

In [ ]:
1